### **Import dependencies**

In [2]:
import collections
import tensorflow as tf
import tensorflow_federated as tff
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import pandas as pd
import numpy as np
from classifiers import create_nn

### **Setup configuration variables**

In [3]:
NUM_CLIENTS = 10
NUM_ROUNDS = 10

### **Preprocess data**

In [10]:
df = pd.read_csv('csv_files/merged_df_with_dates.csv', sep=',')
attack_type = 'no_attack'

y = df['is_malware']
X = df.drop('is_malware', axis=1).select_dtypes(include=['int', 'float']).values

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert the data into Tensors
X_train = tf.constant(X_train, dtype=tf.float32)
y_train = tf.constant(y_train, dtype=tf.int32)
X_test = tf.constant(X_test, dtype=tf.float32)
y_test = tf.constant(y_test, dtype=tf.int32)

# Simulate the clients' data
client_data = []
for i in range(NUM_CLIENTS):
    start = i * len(X_train) // NUM_CLIENTS
    end = (i + 1) * len(X_train) // NUM_CLIENTS
    client_data.append(
        tf.data.Dataset.from_tensor_slices((X_train[start:end], y_train[start:end])).batch(1))


2023-06-10 16:16:15.573213: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


### **Prepare client models**

In [12]:
# Create the TFF version of the model
def model_fn():
    keras_model = create_nn(input_shape=(X.shape[1],), compile=False)
    return tff.learning.models.from_keras_model(
        keras_model,
        input_spec=client_data[0].element_spec,
        loss=tf.keras.losses.BinaryCrossentropy(),
        metrics=[tf.keras.metrics.BinaryAccuracy()]
    )

# Create the federated data
federated_data = [client_data[i] for i in range(NUM_CLIENTS)]

# Create the TFF model and federated learning process
federated_averaging_process = tff.learning.algorithms.build_weighted_fed_avg(
    model_fn,
    client_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=0.02),
    server_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=1.0)
)

def train(federated_averaging_process, num_clients_per_round, num_rounds):
    state = federated_averaging_process.initialize()

    for round_num in range(num_rounds):
        sampled_clients = np.random.choice(range(NUM_CLIENTS), size=num_clients_per_round, replace=False)
        sampled_train_data = [federated_data[i] for i in sampled_clients]

        result = federated_averaging_process.next(state, sampled_train_data)
        state = result.state
        print(result.metrics['client_work']['train'])

train(federated_averaging_process, NUM_CLIENTS, NUM_ROUNDS)


2023-06-10 18:29:45.704490: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2023-06-10 18:29:45.705908: I tensorflow/core/grappler/clusters/single_machine.cc:358] Starting new session
2023-06-10 18:29:45.777509: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2023-06-10 18:29:45.777654: I tensorflow/core/grappler/clusters/single_machine.cc:358] Starting new session
2023-06-10 18:29:46.630004: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2023-06-10 18:29:46.630165: I tensorflow/core/grappler/clusters/single_machine.cc:358] Starting new session
2023-06-10 18:29:46.810798: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2023-06-10 18:29:46.810946: I tensorflow/core/grappler/clusters/single_machine.cc:358] Starting new session


OrderedDict([('binary_accuracy', 0.8847164), ('loss', 0.29796183), ('num_examples', 12569), ('num_batches', 12569)])
OrderedDict([('binary_accuracy', 0.9189275), ('loss', 0.22824213), ('num_examples', 12569), ('num_batches', 12569)])
OrderedDict([('binary_accuracy', 0.9226669), ('loss', 0.21991591), ('num_examples', 12569), ('num_batches', 12569)])


KeyboardInterrupt: 